In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!ln -s /content/drive/MyDrive /mydrive
!ls /mydrive

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height = 224
img_width = 224
batch_size = 64

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    '/mydrive/LSTR/logo/',
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=100,
    validation_split=0.1,
    subset="training")

ds_val = tf.keras.preprocessing.image_dataset_from_directory(
    '/mydrive/LSTR/logo/',
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=100,
    validation_split=0.1,
    subset="validation")

In [ ]:
import numpy as np
y = np.concatenate([y for x,y in ds_train], axis=0)

In [ ]:
val, count = np.unique(y, return_counts=True)

total = np.sum(count)
classes = len(count)

class_weight = {}
for i, value in enumerate(val):
  class_weight[value] = (1/count[i])*(total/classes)

In [ ]:
print(class_weight)

In [ ]:
import tensorflow as tf
from keras import layers
import numpy as np
from tensorflow.keras import initializers
from keras import backend
from keras.engine import training
from keras.applications import imagenet_utils
from keras.layers import VersionAwareLayers
from keras.utils import data_utils
from keras.utils import layer_utils

def VGG16CAM(classes=2, input_tensor=None, input_shape=None,
             weights=None, pooling=None, num_input_channels=1024):

  layers = VersionAwareLayers()
  #tf.keras.backend.set_image_data_format('channels_first')
  input_shape = imagenet_utils.obtain_input_shape(
      input_shape, default_size = 224, min_size=32,
      data_format=backend.image_data_format(),
      require_flatten=True,
      weights=weights)

  if input_tensor is None:
    img_input = layers.Input(shape=input_shape)
  else:
    if not backend.is_keras_tensor(input_tensor):
      img_input = layers.Input(tensor=input_tensor, shape=input_shape)
    else:
      img_input = input_tensor
  #Block 1
  x = layers.Conv2D(64, (3,3), activation='relu',
                    padding='same', name='block1_conv1')(img_input)
  x = layers.Conv2D(64, (3,3), activation='relu',
                    padding='same', name='block1_conv2')(x)
  x = layers.MaxPooling2D((2,2), strides=(2,2), name='block1_pool')(x)
  #Block 2
  x = layers.Conv2D(128, (3,3), activation='relu',
                    padding='same', name='block2_conv1')(x)
  x = layers.Conv2D(128, (3,3), activation='relu',
                    padding='same', name='block2_conv2')(x)
  x = layers.MaxPooling2D((2,2), strides=(2,2), name='block2_pool')(x)
  #Block 3
  x = layers.Conv2D(256, (3, 3), activation='relu',
                    padding='same', name='block3_conv1')(x)
  x = layers.Conv2D(256, (3, 3), activation='relu',
                    padding='same', name='block3_conv2')(x)
  x = layers.Conv2D(256, (3, 3), activation='relu',
                    padding='same', name='block3_conv3')(x)
  x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
  #Block 4
  x = layers.Conv2D(512, (3, 3), activation='relu',
                    padding='same', name='block4_conv1')(x)
  x = layers.Conv2D(512, (3, 3), activation='relu',
                    padding='same', name='block4_conv2')(x)
  x = layers.Conv2D(512, (3, 3), activation='relu',
                    padding='same', name='block4_conv3')(x)
  x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
  #Block 5
  x = layers.Conv2D(512, (3, 3), activation='relu',
                    padding='same', name='block5_conv1')(x)
  x = layers.Conv2D(512, (3, 3), activation='relu',
                    padding='same', name='block5_conv2')(x)
  x = layers.Conv2D(512, (3, 3), activation='relu',
                    padding='same', name='block5_conv3')(x)
  #Added Conv Layers
  x = layers.Conv2D(num_input_channels, (3,3), activation='relu',
                    padding='same', name='CAM_conv')(x)
  #GMP or GAP

  if pooling == 'avg':
    x = layers.GlobalAveragePooling2D()(x)
  elif pooling == 'max':
    x = layers.GlobalMaxPooling2D()(x)

  x = layers.Flatten()(x)
  x = layers.Dense(classes, activation='softmax',
                   use_bias=False,name='prediction')(x)  
  if input_tensor is not None:
    inputs = layer_utils.get_source_inputs(input_tensor)
  else:
    inputs = img_input
  model = training.Model(inputs, x, name='VGG16CAM')
  #Load Weights
  if weights is not None:
    model.load_weights(weights)
  
  return model

In [ ]:
from tensorflow import keras
model = VGG16CAM(classes=2, weights=None, pooling='avg')
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

In [ ]:
import tensorflow as tf

path = '/content/drive/MyDrive/LSTR/CAMSA_3classes'
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = path,
    verbose = 1,
    save_weights_only = False,
    save_best_only=True)

In [ ]:
import tensorflow as tf
new_model = tf.keras.models.load_model(path)
new_model.evaluate(ds_val)

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model(path)
model.evaluate(ds_val)

In [ ]:
model.fit(ds_train, epochs=100, verbose=2, validation_data=ds_val, callbacks=cp_callback, class_weight=class_weight)

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
from keras.engine import training
model_path = '/content/drive/MyDrive/LSTR/CAMSA_3classes'

model = tf.keras.models.load_model(model_path)
intermediate_layer_model = training.Model(inputs=model.input, outputs=model.layers[-4].output)

In [ ]:
def create_cmap(feature_blobs, softmax_weights):
  return np.dot(feature_blobs, softmax_weights)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def plot_cmap(cmap, img_tensor, class_idx, ratio=16):
  for i,_ in enumerate(cmap):
    fig = plt.figure(figsize=(30,6))
    #ax = fig.add_subplot(1, len(class_idx),1)
    for idx in class_idx:
      ax = fig.add_subplot(1, len(class_idx), idx+1)
      img = cmap[i,:,:,idx]
      img = cv2.resize(img, (img.shape[0]*ratio, img.shape[1]*ratio), interpolation=cv2.INTER_LINEAR)
      img = img - np.min(img)
      img = img/np.max(img)
      img = np.uint8(255*img)
      ax.imshow(img_tensor[i])
      ax.imshow(img, cmap='jet', alpha=0.5, interpolation='nearest')
    plt.show()

In [ ]:
import numpy as np

img_path = '/content/drive/MyDrive/LSTR/logo/hybrid/1480647.jpeg'

im = cv2.resize(cv2.imread(img_path), (224,224), interpolation=cv2.INTER_LINEAR)
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
feature_blobs = intermediate_layer_model.predict(np.expand_dims(im, axis=0))
softmax_weights = model.layers[-1].get_weights()[0]
cmap = np.dot(feature_blobs, softmax_weights)

img = cmap[0,:,:,0]
#img = cv2.resize(img, (img.shape[0]*ratio, img.shape[1]*ratio), interpolation=cv2.INTER_LINEAR)
img = img - np.min(img)
img = img/np.max(img)
img = np.uint8(255*img)
print(img)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2

path = '/content/drive/MyDrive/LSTR/logo/hybrid'
img_array = []

for img_file in os.listdir(path):
  img_path = os.path.join(path, img_file)
  image = cv2.imread(img_path)
  image = cv2.resize(image, (224,224), interpolation=cv2.INTER_LINEAR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  img_array.append(image)

img_tensor = tf.convert_to_tensor(img_array)
feature_blobs = intermediate_layer_model.predict(img_tensor)
softmax_weights = model.layers[-1].get_weights()[0]
cmap = create_cmap(feature_blobs, softmax_weights)
plot_cmap(cmap, img_tensor, [0,1,2])

In [ ]:
import matplotlib.pyplot as plt

tmp = np.reshape(cmap[:,:,:,1], -1)
_ = plt.hist(tmp, bins=12)
plt.show()

In [ ]:
import numpy as np

def map_range(left_old, right_old, left_new, right_new, v):
  range_old = right_old - left_old
  range_new = right_new - left_new

  v = (v-left_old)/(range_old)

  v = left_new + v*range_new
  return v

def create_attention_map(text_cmap):
  tmp = np.reshape(text_cmap, -1)
  #tmp[tmp<=0] = 0
  tmp = 1.5*(tmp-np.min(tmp))/(np.max(tmp)-np.min(tmp))
  tmp = np.reshape(tmp, text_cmap.shape)
  return tmp

In [ ]:
import numpy as np
from scipy import spatial
def distance(v1,v2, d_type='cosine'):
  if d_type == 'd1':
    return  np.sum(np.absolute(v1-v2))
  elif d_type =='d2':
    return np.sum((v1-v2)**2)
  elif d_type =='cosine':
    v1 = np.reshape(v1, -1)
    v2 = np.reshape(v2, -1)
    return spatial.distance.cosine(v1,v2)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from keras.engine import training

path = '/content/drive/MyDrive/LSTR/database'
img_array = []

for img_file in os.listdir(path):
  img_path = os.path.join(path, img_file)
  image = cv2.imread(img_path)
  image = cv2.resize(image, (224,224), interpolation=cv2.INTER_LINEAR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  img_array.append(image)

img_tensor = tf.convert_to_tensor(img_array, dtype=np.uint8)
del img_array
feature_blobs = intermediate_layer_model.predict(img_tensor)
softmax_weights = model.layers[-1].get_weights()[0]
cmap = create_cmap(feature_blobs, softmax_weights)
del feature_blobs
del softmax_weights
figure_cmap = cmap[:,:,:,1]
#text_cmap = cmap[:,:,:,2]


vgg16 = VGG16(weights='imagenet')
feature_extractor = training.Model(inputs=vgg16.input, outputs=vgg16.layers[-6].output)
features = feature_extractor.predict(preprocess_input(img_tensor))

feature_with_attention = np.einsum('ijkz,ijk->ijkz', features, figure_cmap)
img_feature_vector = tf.keras.layers.GlobalMaxPool2D()(feature_with_attention)

d = np.zeros((len(img_tensor),len(img_tensor)))
for i in range(len(img_tensor)):
  v1 = img_feature_vector[i]
  d[i,i] = 0
  for j in range(i+1, len(img_tensor)):
    v2 = img_feature_vector[j]
    d[i,j] = distance(v1, v2, d_type='cosine')
    d[j,i] = d[i,j]

In [ ]:
n=5
for i in range(len(img_tensor)):
  fig = plt.figure(figsize=(30,6))
  ax = fig.add_subplot(1, n+1, 1)
  ax.imshow(img_tensor[i])
  tmp = d[i]
  query = np.argsort(tmp)
  for j in range(n):
    ax = fig.add_subplot(1, n+1, j+2)
    ax.imshow(img_tensor[query[j+1]])
  plt.show()

In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from keras.engine import training
import matplotlib.pyplot as plt

#Input paths
query_path = '/content/drive/MyDrive/LSTR/logo/shape/mickey-mouse-character-vector-400x400.png' #the image we want to query
database_path = '/content/drive/MyDrive/LSTR/logo/shape'
model_path = '/content/drive/MyDrive/LSTR/CAMSA_3classes'

#Class activation map model
model = tf.keras.models.load_model(model_path)
intermediate_layer_model = training.Model(inputs=model.input, outputs=model.layers[-4].output)

#Save database into tensor
query_image = cv2.imread(query_path)
query_image = cv2.resize(query_image, (224,224), interpolation=cv2.INTER_LINEAR)
query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
query_tensor = tf.convert_to_tensor(np.expand_dims(query_image, axis=0), dtype=np.uint8)

img_array = []
for img_file in os.listdir(database_path):
  img_path = os.path.join(database_path, img_file)
  image = cv2.imread(img_path)
  image = cv2.resize(image, (224,224), interpolation=cv2.INTER_LINEAR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #as opencv read image as BGR
  img_array.append(image)
database_tensor = tf.convert_to_tensor(img_array, dtype=np.uint8)

#Create Class Activation Map
query_feature_blob = intermediate_layer_model.predict(query_tensor) #shape = [1, 14,14,1024]
feature_blobs = intermediate_layer_model.predict(database_tensor) #shape = [bs, 14,14,1024] (1024 channels, each is a 14x14 image)

softmax_weights = model.layers[-1].get_weights()[0] #shape = [1024, 3] (1024 weights for 3 classes)

query_figure_cmap = create_cmap(query_feature_blob, softmax_weights)[:,:,:,1] #shape = [1, 14, 14], change the last index to choose your cmap
figure_cmap = create_cmap(feature_blobs, softmax_weights)[:,:,:,1] #shape = [bs, 14, 14]

#Create feature extractor
vgg16 = VGG16(weights='imagenet')
feature_extractor = training.Model(inputs=vgg16.input, outputs=vgg16.layers[-6].output) #output of block5_conv3
#Save features into tensor
query_feature = feature_extractor.predict(preprocess_input(query_tensor)) #shape = [1,14,14,512] 
features = feature_extractor.predict(preprocess_input(database_tensor)) #shape = [bs, 14,14,512] (512 channels, each is a 14x14 image)

#Attention & MAC
query_with_attention = np.einsum('ijkz,ijk->ijkz', query_feature, query_figure_cmap)
feature_with_attention = np.einsum('ijkz,ijk->ijkz', features, figure_cmap)
query_feature_vector = tf.keras.layers.GlobalMaxPool2D()(query_with_attention)
img_feature_vector = tf.keras.layers.GlobalMaxPool2D()(feature_with_attention)

#Create Distance Vector (einsum is faster than normal matrix multiplication)
d = np.zeros(len(database_tensor))
v1 = query_feature_vector
for j in range(len(database_tensor)):
  v2 = img_feature_vector[j]
  d[j] = distance(v1,v2, d_type='cosine')

#Return Query
n=5
fig = plt.figure(figsize=(30,6))
ax = fig.add_subplot(1, n+1, 1)
ax.imshow(query_image)
index = np.argsort(d)
for j in range(n):
  ax = fig.add_subplot(1, n+1, j+2)
  ax.imshow(database_tensor[index[j]])
plt.show()

In [ ]:
import tensorflow as tf
from keras import layers
import numpy as np
from tensorflow.keras import initializers
from keras import backend
from keras.engine import training
from keras.applications import imagenet_utils
from keras.layers import VersionAwareLayers
from keras.utils import data_utils
from keras.utils import layer_utils

def UNet():
  layers = VersionAwareLayers()
  #tf.keras.backend.set_image_data_format('channels_first')
  input_shape = imagenet_utils.obtain_input_shape(
      input_shape, default_size = 224, min_size=32,
      data_format=backend.image_data_format(),
      require_flatten=True,
      weights=weights)

  if input_tensor is None:
    img_input = layers.Input(shape=input_shape)
  else:
    if not backend.is_keras_tensor(input_tensor):
      img_input = layers.Input(tensor=input_tensor, shape=input_shape)
    else:
      img_input = input_tensor
  #Downsampling block
  x = layers.Conv2D(64, (3,3), activation='relu', name='Down_Conv1', padding='same')(img_input)